### 批量运行质控任务

In [8]:
import openpyxl
import boto3
import re

##################################################################
#excel中保存输入文件信息
wb = openpyxl.load_workbook('sample-info.xlsx')
sheet = wb['metagenomic']
#改成你自己的账户ID
account='691348495649'

##################################################################
#任务脚本
script = '/scripts/qc.sh'
#任务信息保存在DynamoDB表中
dbtable = 'metagenomic_qc'
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table(dbtable)
#输入文件存储在该S3桶中
bucket = 'metagenomic-' + account + '-cn-northwest-1'

batch = boto3.client('batch')
#Batch任务定义
jobdef='jd-qc-test:5'
#Batch任务队列
jobqueue='q-qc-test'

for row in sheet.iter_rows(min_row=2):
    sample = row[0].value
    threads=16
    memory='31500'
    
    # init ddb table
    table.put_item(
       Item={
            'sample': sample,
            'threads': threads,
            'qc_time': 0,
            'status': 0
        }
    )
    
    # submit jobs
    jobname = sample + '-' + str(threads)
    print('jobname: ', jobname, 'queue: ', jobqueue)
    params = {
        'script':script,
        'sample':sample,
        'threads':str(threads),
        'bucket':bucket,
        'dbtable':dbtable
    }
    resourceRequirements = [
        {
            'value':str(threads),
            'type':'VCPU'
        },
        {
            'value':memory,
            'type':'MEMORY'
        },
    ]
    response = batch.submit_job(
        jobName=jobname,
        jobQueue=jobqueue,
        jobDefinition=jobdef,
        parameters=params,
        containerOverrides={
            'resourceRequirements':resourceRequirements
        }
    )
    print(response)

jobname:  SRR10389008-16 queue:  q-qc-test
{'ResponseMetadata': {'RequestId': '65b7b454-3d69-4301-b235-1adf4257ed35', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 03 Oct 2023 08:41:18 GMT', 'content-type': 'application/json', 'content-length': '172', 'connection': 'keep-alive', 'x-amzn-requestid': '65b7b454-3d69-4301-b235-1adf4257ed35', 'access-control-allow-origin': '*', 'x-amz-apigw-id': 'MN4DSGz95PgF9Dw=', 'access-control-expose-headers': 'X-amzn-errortype,X-amzn-requestid,X-amzn-errormessage,X-amzn-trace-id,X-amz-apigw-id,date', 'x-amzn-trace-id': 'Root=1-651bd3ae-753e3eba7b03a53263a0748d'}, 'RetryAttempts': 0}, 'jobArn': 'arn:aws-cn:batch:cn-northwest-1:691348495649:job/a943ee3b-5f6e-444a-a758-0fc1d38e7a3e', 'jobName': 'SRR10389008-16', 'jobId': 'a943ee3b-5f6e-444a-a758-0fc1d38e7a3e'}
jobname:  SRR10389009-16 queue:  q-qc-test
{'ResponseMetadata': {'RequestId': '26ece85b-a29b-48b9-9c36-89bcf6836bdc', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 03 Oct 2023 08:41:18